In [ ]:
import path_ocpy


In [ ]:
from ocpy import DDPSolver, iLQRSolver
import ocp_hexacopter

import numpy as np


In [ ]:
ocp = ocp_hexacopter.ocp_hexacopter_unconstrained(constrain='barrier')
# ocp = ocp_hexacopter.ocp_hexacopter_unconstrained(constrain='penalty', mu=1e0)
# ocp = ocp_hexacopter.ocp_hexacopter()


In [ ]:
# solver = DDPSolver(ocp, init=False)
solver = iLQRSolver(ocp, init=False)


In [ ]:
# If needed, reset parameters and set solution guess.
n_x = ocp.get_n_x()
n_u = ocp.get_n_u()
T = ocp.get_T()
N = ocp.get_N()
t0 = ocp.get_t0()
x0 = ocp.get_x0()

# t0 = 0.0
# x0 = np.array([0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# solver.set_initial_condition(t0=t0, x0=x0)

# T = 5.0
# N = 200
# solver.set_horizon(T=T, N=N)

# solver.reset_guess()
us_guess = np.array([[1, 1, 1, 1, 1, 1] for i in range(N)])
solver.set_guess(us_guess=us_guess)


In [ ]:
solver.init_solver()

In [ ]:
# set hyperparameters.
solver.set_line_search_param(alpha_min=1e-4, r_alpha=0.8, enable_line_search=True)
solver.set_regularization_param(
    gamma_init=0.0, r_gamma=5.0, gamma_min=0.0, gamma_max=1e6, fix_gamma=False
)
solver.set_stop_tol(stop_tol=1e-5)
solver.set_iters_param(min_iters=0, max_iters=1000)


In [ ]:
solver.solve(
    result=True, log=True, plot=True
)


In [ ]:
# Visualize
from ocpy.animator import HexacopterAnimator
animator = HexacopterAnimator(solver.get_log_directory(), ocp.get_ocp_name())
animator.generate_animation(save=False, skip_rate=3)


In [ ]:
import matplotlib.pyplot as plt
import math

result = solver.get_result()

gamma_hist = result['gamma_hist']
plt.plot(gamma_hist)
plt.title('gamma')
plt.show()

alpha_hist = result['alpha_hist']
plt.plot(alpha_hist)
plt.title('alpha')
plt.show()
print('average alpha:',sum(alpha_hist / (len(alpha_hist) - 1)))

cost_hist = result['cost_hist']
plt.plot(cost_hist)
plt.title('cost')
plt.show()
